<a href="https://colab.research.google.com/github/NikitaKProjects/Fake-News-Detection-System-/blob/main/Fake_News_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import re
import string

In [ ]:
data_fake = pd.read_csv("/Fake.csv")
data_true = pd.read_csv("/True.csv")
data_fake.head()
data_true.head()

In [ ]:
data_fake["class"]=0
data_true["class"]=1

In [ ]:
data_fake.shape, data_true.shape

In [ ]:
data_fake_manual_testing = data_fake.tail(10)
for i in range(23480,23470,-1):
  data_fake.drop([i],axis=0,inplace=True)

data_true_manual_testing = data_true.tail(10)
for i in range(21416,21406,-1):
  data_true.drop([i],axis=0,inpace=True)

In [ ]:
data_fake_manual_testing['class']=0
data_true_manual_testing['class']=1

In [ ]:
data_merge = pd.concat([data_fake,data_true],axis=0)
data_merge.head(10)

In [ ]:
data = data_merge.drop(['title','subject','date'],axis=1)

In [ ]:
def wordopt(text):
    # Convert text to lowercase
    text = text.lower()

    # Remove text within square brackets
    text = re.sub(r'\[.*?\]', '', text)

    # Remove all non-word characters (punctuation, etc.)
    text = re.sub(r"\W", " ", text)

    # Remove URLs
    text = re.sub(r'https?://\S+|www\.\S+', '', text)

    # Remove HTML tags
    text = re.sub(r'<.*?>+', '', text)

    # Remove punctuation using re.escape for safety
    text = re.sub(f'[{re.escape(string.punctuation)}]', '', text)

    # Remove new lines
    text = re.sub(r'\n', '', text)

    # Remove words containing numbers
    text = re.sub(r'\w*\d\w*', '', text)

    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()

    return text

In [ ]:
data['text']= data['text'].apply(wordopt)

In [ ]:
x=data['text']
y=data['class']

In [ ]:
x_train, x_test, y_train,y_test = train_test_split(x,y,test_size=0.25)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

In [ ]:
from sklearn.linear_model import LogisticRegression

LR = LogisticRegression()
LR.fit(xv_train,y_train)

In [ ]:
pred_lr = LR.predict(xv_test)
LR.score(xv_test,y_test)
print(classification_report(y_test,pred_lr))

In [ ]:
from sklearn.tree import DecisionTreeClassifier

DT = DecisionTreeClassifier()
DT.fit(xv_train,y_train)

In [ ]:
pred_dt = DT.predict(xv_test)
DT.score(xv_test,y_test)
#print(classification_report(y_test,pred_dt))

In [ ]:

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report, accuracy_score

GBC = GradientBoostingClassifier()
GBC.fit(xv_train, y_train)
pred_gbc = GBC.predict(xv_test)
accuracy_gbc = GBC.score(xv_test, y_test)
print("Gradient Boosting Classifier Accuracy: {:.2f}%".format(accuracy_gbc * 100))
print(classification_report(y_test, pred_gbc))

In [ ]:
from sklearn.ensemble import RandomForestClassifier

RFC = RandomForestClassifier()
RFC.fit(xv_train, y_train)
pred_rfc = RFC.predict(xv_test)
accuracy_rfc = RFC.score(xv_test, y_test)
print("Random Forest Classifier Accuracy: {:.2f}%".format(accuracy_rfc * 100))
print(classification_report(y_test, pred_rfc))


In [ ]:
def output_label(n):
    if n == 0:
        return "Fake_News"
    elif n == 1:
        return "Not a Fake_News"

def manual_testing(news):
    testing_news = {"text": [news]}
    new_def_test = pd.DataFrame(testing_news)
    new_def_test["text"] = new_def_test["text"].apply(wordopt)
    new_x_test = new_def_test["text"]
    new_xv_test = vectorization.transform(new_x_test)

    # Predictions
    pred_LR = LR.predict(new_xv_test)
    pred_DT = DT.predict(new_xv_test)
    pred_GBC = GBC.predict(new_xv_test)
    pred_RFC = RFC.predict(new_xv_test)

    return print("\n\nLR Prediction: {} \nDT Prediction: {} \nGBC Prediction: {} \nRFC Prediction: {}".format(
        output_label(pred_LR[0]),
        output_label(pred_DT[0]),
        output_label(pred_GBC[0]),
        output_label(pred_RFC[0])
    ))


In [ ]:
news =str(input())
manual_testing(news)